# Code for simulating disk packings on cones

By Jessica H. Sun and Abigail Plummer

Change the following variable to point to the directory containing files from the [Harvard DataVerse repository](https://doi.org/10.7910/DVN/ZQ4BUR).


In [ ]:
parent_directory='/Users/jessicasun/Downloads/'

In [ ]:
import sys
import numpy as np
import random
import os
import zipfile
import io
import pandas as pd
from math import atan2, cos, sin

In [ ]:
inputzip = zipfile.ZipFile("simulation_inputs/run_dir_list_022322_0-99.npy.zip")

In [ ]:
inputzip.infolist()

In [ ]:
bytes = inputzip.read(inputzip.infolist()[0])

In [ ]:
#define a class that all of our functions live inside.
class ConeBennett:
    """init is a constructor, which allows us to start a new trial with all the right attributes that we might need.
    this first parentheses has the default variables in it
    if i just called f=ConeBennett(), for example, it would create an instance of the simulation with all of these default params
    if instead I called f=ConeBennett(N=12), it would have all of the default values except for N, which would be 12
    """
    def __init__(self, Nmax=500, ctheta=np.pi/8.0, cinit=10,itheta=0):
        self.ctheta=ctheta

        # number of particles that we will place on the surface IN ADDITION to the triangular seed
        self.Nmax=Nmax
        #distance between particles. For now, we leave it fixed at 1, so all distances are measured in terms of this
        self.a0=1
        
        #orientation angle
        self.itheta=itheta
        
        #initialization height of seed
        self.cinit=cinit
        if self.ctheta!=0:
            self.yinit=cinit/self.ctheta #arc length / sector angle = radius of sector
        else: #for a cylinder, assume semi-infinite cone..
            self.yinit=cinit/0.002 
            self.Lx=self.cinit #width of simulation box, which on x-axis is (-Lx/2,+Lx/2]        
        #define a tolerance for when crystals meet each other again-- what will we accept as neighbors?
        #this code is not designed for anything besides hard spheres, so things will probably go wrong if this is large.
        #if tolerance is too small, and we use a ring initial condition, we can run into issues with discretization. Can (and should) calculate this threshold by comparing arc btwn two pts to shortest path
        self.tol=0.0001

        #define the variable we will use to keep track of how many particles we have placed. 
        self.n=0

        
        #height of the cone (radius of circle that the cone is a rolled up sector of)
        self.R=self.yinit+20

        #define adjacency matrix, which will be symmetric. Each row will represent a particle. in row i, col j is 1 if there exists a bond between i and j and 0 otherwise. 
        #each particle now effectively has a `name', which is the row number representing that particle.
        self.adj=np.zeros((self.Nmax,self.Nmax))

        #another matrix, N rows, 2 columns. Each row index here is also a row in the adjacency matrix, and we can go back and forth looking up positions and neighbors. 
        self.pos=np.zeros((self.Nmax, 2))

        #create a container to hold the candidate states. We also use a list here. 
        #we will keep track of the location in xy and the energy here.
        self.candidates=[]

        #a list of sites that have open neighbors 
        self.edgeSites=[]


    #save particle positions in x y in the same directory as the code
    def saveParticles(self):
        fileout='particles.dat'
        outfile=self.pos[:self.n,]
        #you can change the number of decimal places output here 
        np.savetxt(fileout,np.c_[outfile],fmt='%.6g')

    def saveMov(self):
        fileout='mov/particles'+str(self.n)+'.dat'
        outfile=self.pos[:self.n,]
        #you can change the number of decimal places output here 
        np.savetxt(fileout,np.c_[outfile],fmt='%.6g')

    def saveAdj(self):
        fileout='adj.dat'
        outfile=self.adj[:self.n,:self.n]
        np.savetxt(fileout,np.c_[outfile],fmt='%.6g')

    #periodic boundaries act on a position and return a position.
    #periodic in the angle (if cylinder, periodic in x since cylinder can be unrolled into rectangle)
    #RHS is included in domain
    def pbc(self,p0): #moves point into the middle simulation sector
        if self.ctheta!=0: #if sector angle > 0
            #atan2(y,x) gives an unambigous radians value between -pi and pi. should be in a window surrounding -pi/2
            angle=atan2(p0[1],p0[0])
            r=np.sqrt(p0[0]**2+p0[1]**2)
            #include the RHS in the domain
            if angle>(-np.pi/2.0+self.ctheta/2.0):
                angle=angle-np.floor((abs(angle-(-np.pi/2))+self.ctheta*0.5)/self.ctheta)*self.ctheta
            if angle<=(-np.pi/2.0-self.ctheta/2.0):
                angle=angle+np.floor((abs(angle-(-np.pi/2))+self.ctheta*0.5)/self.ctheta)*self.ctheta
            #now convert back to x,y coordinates
            return [r*cos(angle),r*sin(angle)]
        else: #if cylinder... (since bennett analysis is very sensitive, it's better to use pbc of a true cylinder)
            x,y=p0[0],p0[1]
            if x>self.Lx/2.0:
                x=x-np.floor((x+self.Lx*0.5)/(self.Lx))*self.Lx
            if x<=-self.Lx/2.0:
                x=x+np.floor(abs(x-self.Lx*0.5)/self.Lx)*self.Lx
            return [x,y]            
    def periodicImages(self,x1,y1): #moves point in the niddle simulation sector into L & R image positions
        if self.ctheta!=0:
            angle=atan2(y1,x1)
            r=np.sqrt(x1**2+y1**2)
            #move to the image sector to the right
            angleR=angle+self.ctheta
            #move to image sector to the left
            angleL=angle-self.ctheta
            #output a list of the positions of the right image and left image points
            return [r*cos(angleR), r*sin(angleR), r*cos(angleL), r*sin(angleL)]
        else:
            x,y=x1,y1
            #move to the image sector to the right
            xR=x1+self.Lx
            #move to image sector to the left
            xL=x1-self.Lx
            #output a list of the positions of the right image and left image points
            return [xR,y1,xL,y1]

    #we need something that returns the minimum distance between two particles, respecting periodic boundaries.
    def findDistance(self,p0, p1):
        #hold point 0 fixed, and compute its distance with both the particle in the domain and the two periodic images. are there addtl complications for a cone?
        pIm=self.periodicImages(p1[0],p1[1])
        d1=np.sqrt((p0[0]-p1[0])**2+(p0[1]-p1[1])**2)
        d2=np.sqrt((p0[0]-pIm[0])**2+(p0[1]-pIm[1])**2)
        d3=np.sqrt((p0[0]-pIm[2])**2+(p0[1]-pIm[3])**2)
        return min([d1,d2,d3])

    #also return the image point used to compute distances
    def findDistanceandIm(self,p0, p1):
        #hold point 0 fixed, and compute its distance with both the particle in the domain and the two periodic images. are there addtl complications for a cone?
        pIm=self.periodicImages(p1[0],p1[1])
        d1=np.sqrt((p0[0]-p1[0])**2+(p0[1]-p1[1])**2)
        d2=np.sqrt((p0[0]-pIm[0])**2+(p0[1]-pIm[1])**2)
        d3=np.sqrt((p0[0]-pIm[2])**2+(p0[1]-pIm[3])**2)
        #not mutually exclusive- is this an issue for us?
        if d1<=d2 and d1<=d3:
            return [d1, p1[0], p1[1]]
        if d2<=d1 and d2<=d3:
            return [d2, pIm[0], pIm[1]]
        if d3<=d1 and d3<=d2:
            return [d3, pIm[2], pIm[3]]

    #return a bool that says whether or not we are within the boundary, given an i,j point
    #ONLY TEST if it is out of bounds in the y direction, because these are fixed. Put PBCs on the x direction.  
    def boundary(self,pos):
        if np.sqrt(pos[0]**2+pos[1]**2)<self.R: 
            return True
        else:
            return False

    #this method takes two pre-existing points and finds two other points that are a distance a0 from BOTH of them. Only works when the distance between the two points is between 1 and 2
    def findMutualSites(self, dist, x0,y0,x1,y1):
        midpt=[(x0+x1)/2.0, (y0+y1)/2.0]
        perpbisector=[(y1-y0), -(x1-x0)]
        altitude=np.sqrt(self.a0**2-(dist/2.0)**2)
        pos1=self.pbc([midpt[0]+perpbisector[0]/(1.0*dist)*altitude, midpt[1]+perpbisector[1]/(1.0*dist)*altitude])
        pos2=self.pbc([midpt[0]-perpbisector[0]/(1.0*dist)*altitude, midpt[1]-perpbisector[1]/(1.0*dist)*altitude])
        return [pos1, pos2]

    #consider the two points that connect to our candidate state-- is one of them already 'filled'?
    #this (hopefully?) saves computational time-- we check the mostly likely candidates for overlap first, and this hopefully saves us from having to iterate over everything
    def noPreexistingPart(self,l, m, pos1, pos2):
        #define two bools, and they will be true if there is no overlap from a preexisting neighbor, and false otherwise
        pos1Bool=True
        pos2Bool=True	
        #we sum up adjacency matrix to find neighbors that they both share. Anywhere that there is a 2, it is a mutual neighbor. 
        mutuals=self.adj[l,]+self.adj[m,]
        #get a np array of anywhere that it sums to two 
        mutualIdx=(np.where(mutuals==2))[0]
        #use a list comprehension on a numpy array (a little sketchy lol but seems to work)
        #this for loop says-- look at the pre-existing particles that share an edge with l and m particles. Are either of the proposed candidate states overlapping with this? 
        for j in mutualIdx:
            neighbpos=self.pos[mutualIdx[0],]
            #we should't need to run the fancy findDist here (that takes into account PBCs) bc we're looking to see if something is actually in the exact spot. 
            #of course...it is possible that there is some extremely weird edge case where the center is precisely on the boundary...
            #I guess we might as well do it, and can probably remove this if our code is slow without too much heartache (worst case scenario, we add a redundant point)

            #like in noDuplicates, we hard code in a value instead of using tol bc this is asking the question-- does this EXACT point already exist? 
            if abs(self.findDistance(pos1, neighbpos))<0.001:
                pos1Bool=False
            if len(pos2)>0:
                if abs(self.findDistance(pos2, neighbpos))<0.001:
                    pos2Bool=False
        return pos1Bool, pos2Bool

    #findEnergy now returns a bondlist, so that we can easily update the adjacency matrix. 
    def findEnergy(self, candpos):
        en=0
        bondlist=[]
        #note that the candidate state is not part of edgeSites, so shouldn't have redundancy. The two neighbors that we used to generate candpos SHOULD be in edgeSites. 
        for idx in self.edgeSites:
            dist=self.findDistance(candpos, self.pos[idx,])
            if abs(dist-self.a0)<self.tol:
                en=en-1
                bondlist.append(idx)
            #energy penalty if it overlaps
            if dist-self.a0<(-1*self.tol):
                en=en+100
        return en, bondlist

    #returns true if candidatepos with bondlist is not already in self.candidates list. Only add it if it's not already there. 
    def noDuplicates(self, candpos, bondlist):
        duplicateBool=True
        for j in self.candidates:
            if set(bondlist)==set(j[2]):
                #we don't use tol here, because this is not about overlap-- we want to remove duplicate candidates
                if abs(self.findDistance(candpos, j[0]))<0.001:
                    duplicateBool=False
        return duplicateBool

    #return a num so we know IF something was added, so we can decide whether or not to keep it. 
    #as you add, check whether or not it is already apart of candidates. This way, we don't need to consider duplicate removal later on. 
    def genCandidatesforPair(self, l, m):
        dist, x1, y1=self.findDistanceandIm(self.pos[l], self.pos[m])
        x0,y0=self.pos[l]
        #two cases to consider here.
        if abs(dist-2*self.a0)<self.tol:
            #if the base pair particles are exactly 2 apart, the only candidate lies directly in between them. average the points, pbc wrapper
            candpos=self.pbc([(x0+x1)/2.0, (y0+y1)/2.0])
            #make sure there isn't already a particle there	
            if (self.noPreexistingPart(l,m,candpos, [])[0]) and self.boundary(candpos):
                #measure the energy, keep track of new connections. Only look through edgeSites-- i think this is okay, because we already checked for conflicts with
                #sites that could possibly have 6 neighbors (would have to be perfectly crystalline, and in the right position, otherwise, it's in edgeSites). Is this right?
                en, bondlist=self.findEnergy(candpos)
                if en<-1 and self.noDuplicates(candpos, bondlist):
                    self.candidates.append([candpos, en, bondlist])
        if abs(dist-self.a0)<self.tol or (dist>self.a0 and dist<(2*self.a0) and abs(dist-2*self.a0)>self.tol):
            #if the pair is exactly 1 apart, we have a crystalline-like packing problem, and we can check neighbors to speed up computation. if it is between 1 and 2 apart, 
            #we still generate new candidates by solving for the vertices of a rhombus. 
            #question- is there ever a time that a pair of hard spheres can share more than 2 neighbors? I don't think so in flat space, but these
            #arguments will have to change when we move to curved space.
            candpos=self.findMutualSites(dist,x0,y0,x1,y1)
            boolList=self.noPreexistingPart(l,m,candpos[0],candpos[1])	
            for idx,val in enumerate(boolList):
                if val and self.boundary(candpos[idx]):
                    en, bondlist=self.findEnergy(candpos[idx])
                    if en<-1 and self.noDuplicates(candpos[idx], bondlist):
                        self.candidates.append([candpos[idx], en, bondlist])

    def analyzeInitSeed(self):
    #append the initial seed to edgeSites(assumes that we don't initialize with any instances of 6 fold coordination, but even if we do, nothing major goes wrong)
        self.edgeSites=list(range(self.n))
    #first, set adjacency matrix, bc next step (generating initial candidates) depends on this.
    #go through each pair of particles once (i >j, i and j less than n)
        for j in range(self.n):
            for k in range((j+1), self.n):
                dist=self.findDistance(self.pos[j], self.pos[k])
                if abs(dist-self.a0)<self.tol:
                    self.adj[j,k]=self.adj[k,j]=1
    #now, go through and generate the possible candidate states. This is setup so that we only consider PAIRS. 
        for l in range(self.n):
            for m in range((l+1), self.n):
                self.genCandidatesforPair(l,m)
    def rotate(self, point, theta):
    #apply rotation matrix to seed
        x=point[0]
        y=point[1]
        rot_point=[x*np.cos(theta)-y*np.sin(theta),x*np.sin(theta)+y*np.cos(theta)]
        return rot_point

    #put an initial triangular seed somewhere
    def initialize(self):
        #initialize system with whatever the desired pattern is. We can do a single triangle, or this is where we would code in a ring at a specified $r$.

        #Be careful setting initial conditions, because there's no error checking here- it will let you place things outside the periodic boundaries
        #also, you will have to add additional stuff if you want to do a non-convex IC, because if there will be duplicate candidate states generated (aka
        # two distinct pairs that suggest the same new spot), this will also cause problems. 
        ninit=3 # initialize triangular seed
        
        #place initial points in triangle with one point on origin at all times
        triangle0=np.array([0,0])
        triangle1=np.array([1,0])
        triangle2=np.array([0.5,np.sqrt(3)/2.0])
        
        #apply rotation matrix - rotate seed by orientation angle itheta
        rot_triangle0=self.rotate(triangle0,self.itheta)
        rot_triangle1=self.rotate(triangle1,self.itheta)
        rot_triangle2=self.rotate(triangle2,self.itheta)
        
        #this results in a seed that rotates about its centroid but
        self.pos[0,]=[rot_triangle0[0], rot_triangle0[1]-self.yinit]
        self.pos[1,]=[rot_triangle1[0],rot_triangle1[1]-self.yinit]
        self.pos[2,]=[rot_triangle2[0], rot_triangle2[1]-self.yinit]
#         #initialize a ring of points. 
#         #set the ring at halfway point, composed of 10 particles. 
#         ninit=12
#         #solve for the correct radius. 
#         r=ninit/self.ctheta
#         for j in range(ninit):
#             angle=-np.pi/2.0+self.ctheta/2.0-j*self.ctheta/(1.0*ninit)
#             self.pos[j,]=[r*cos(angle), r*sin(angle)]

        #increment n by the number of particles that you just added. When n=Nmax, this is done. 
        self.n=self.n+ninit

        #confirm that initial seed is within the domain
        for i in range(self.n):
            if not self.boundary(self.pos[i]):
                print('initial seed is outside of domain')

        #generate the initial candidate list, based on the starting seed, and fill in the adjacency matrix
        self.analyzeInitSeed()
        #seed the random number generator so it is 'deterministically random' data. Comment out if you want 'truly' random. 
#         random.seed(10)

    #returns an index OF THE CANDIDATES LIST that we are going to choose
    def selectNewSite(self):
        #if we have no remaining candidates, throw an error. 
        if len(self.candidates)==0:
            self.saveParticles()
            self.saveAdj()
            raise Exception('no acceptable candidates')
        #make a list of the energies
        envalues=[x[1] for x in self.candidates]
        minval=min(envalues)
        #find all places where the minimum is-- indices is a list of indices
        indices = [idx for idx, x in enumerate(envalues) if x == minval]
        if len(indices)>1:
            sel=random.choice(indices)
        if len(indices)==1:
            sel=indices[0]
        return sel

    def reCalcCandEnergy(self):
    # need to make a dummy list to keep track of who doesn't overlap with our new particle wedged in there. Otherwise, we'd have particles that were ~100 energy in candidates list
        stillAccessible=[]
        for idx, val in enumerate(self.candidates):
            self.candidates[idx][1], self.candidates[idx][2]=self.findEnergy(self.candidates[idx][0])
            #if no overlap was created, we keep this as a candidate.
            if self.candidates[idx][1]<-1:
                stillAccessible.append(self.candidates[idx])
        self.candidates=stillAccessible


    def addParticle(self):
        #find the index of the candidate that we want to place a particle on
        sel=self.selectNewSite()
        newpos, en, bondlist=self.candidates[sel]

        #remove this particle from the candidate list- we cannot pick it again
        del self.candidates[sel]
        #add to the position list
        self.pos[self.n]=newpos

        #add the appropriate bonds in bondlist
        for j in bondlist:
            self.adj[self.n,j]=self.adj[j,self.n]=1

        #This gets kind of wacky when tol gets too high... I think because sometimes candidates are allowed to go through that are in the interior, and then
        #they can spread. As long as tol is reasonably low though, this isn't an issue, and you get the same results and 3x speed up. If you're not sure
        #comment it out. 
        #for j in bondlist:
        #	if np.sum(self.adj[j,])==6:
        #		self.edgeSites.remove(j)
        #	if np.sum(self.adj[j,])>6:
        #		raise Exception('too many neighbors')

        #append our new site to edgeSites with the right condition. 
        #if len(bondlist)<6:
        #	self.edgeSites.append(self.n)

        self.edgeSites.append(self.n)

        #recalculate the energy of the candidate states (in case they were affected by new particle addition). So have to append n to endsites first here.
        self.reCalcCandEnergy()

        #add new candidate states touching new particle--we could find the most likely ones by just using bondlist again, but if we want to also reach
        #ones that may be from far away crystals, for ex, we need to look more broadly.

        #consider each pair of particles in the whole system, except for the final entry (which we know to be self.n because we just added it. )
        #generate new candidates. genCandidatesforPair screens for duplicates itself.
        for j in self.edgeSites[:-1]:
            self.genCandidatesforPair(j, self.n)

        #now that we have fully 'added the particle' we increment n (this relative ordering of steps keeps the indexing right.)
        self.n=self.n+1

    #pull it all together!
    def run(self):
        try:
            self.initialize()
            while self.n<self.Nmax:
                try:
                    self.addParticle()
        #             self.saveMov()
                except: #likely Nmax > dense packing n so break when no more particles can be placed
                    print('break at n='+str(self.n),end=' | ')
                    break
            self.saveParticles()
            self.saveAdj()
        except:
            print('error')


# In[ ]:


def get_params_dict(run_dir):
    keys=[]
    values=[]
    for params in run_dir.split('_'):
        key,value=params.split('-')
        keys+=[key]
        values+=[value]
    params_dict=dict(zip(keys,values))
    return params_dict

#input_dir is directory containing .npy file (experiment parameters)
input_dir=parent_directory+'cone-disk-packings-main/simulation/simulation_inputs/'
os.chdir(input_dir)
run_dir_list=np.load('run_dir_list_022323_0-2cinit0-12.npy')

#input a job number from command line to process a chunk of run_dir_list, or comment out to process all at once
# job_num=int(sys.argv[1])
# run_dir_list=run_dir_list[(job_num-1)*492:job_num*492]

#create datfiles subfolder in input_dir
output_dir=input_dir+'/datfiles'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

counter=0
total=len(run_dir_list)
for run_dir in run_dir_list:
    counter+=1
    os.chdir(output_dir)
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    if not (os.path.exists(run_dir+'/particles.dat'))&(os.path.exists(run_dir+'/adj.dat')): #in the future, edit this to check to make sure adj and particles aren't empty (indicates file writing was interrupted)
        os.chdir(output_dir+'/'+run_dir)
        params_dict=get_params_dict(run_dir)
        
        model=ConeBennett(int(float(params_dict['Nmax'])),float(params_dict['cthetadeg'])/180*np.pi,float(params_dict['cinit']),float(params_dict['ithetadeg'])/180*np.pi)
        model.run()
        print(run_dir+' done! ('+str(counter)+' / '+str(total)+')')

#         output: \n particles.dat: (x,y) coordinates where indices are in order of placement \n adj.dat: adjacency matrix for particle indices'


# In[ ]: